In [1]:
import os
class_names = ['dup', 'giu', 'nha', 'phai', 'trai']
states = [12, 19, 9, 12, 12]
# số trạng thái bằng số âm x3 - số âm = số âm thực sự + 1 khoảng lặng
length = 0
for d in class_names:
    length += len(os.listdir("data/" + d))
print(length)
!pip install librosa
!pip install hmmlearn

239


In [2]:
import librosa
import numpy as np
import os
import math
from sklearn.cluster import KMeans
import hmmlearn.hmm

def get_mfcc(file_path):
    y, sr = librosa.load(file_path) # read .wav file 22050
    hop_length = math.floor(sr*0.010) # 10ms hop
    win_length = math.floor(sr*0.025) # 25ms frame
    # mfcc is 12 x T matrix
    mfcc = librosa.feature.mfcc(
        y, sr, n_mfcc=12, n_fft=1024,
        hop_length=hop_length, win_length=win_length)
    # substract mean from mfcc --> normalize mfcc
    mfcc = mfcc - np.mean(mfcc, axis=1).reshape((-1,1)) 
    # delta feature 1st order and 2nd order
    delta1 = librosa.feature.delta(mfcc, order=1)
    delta2 = librosa.feature.delta(mfcc, order=2)
    #bổ sung vào slide + báo cáo
    # X is 36 x T
    X = np.concatenate([mfcc, delta1, delta2], axis=0) # O^r
    # return T x 36 (transpose of X)
    return X.T # hmmlearn use T x N matrix


In [3]:
all_data = {}
all_labels = {}
for cname in class_names:
    file_paths = [os.path.join("data", cname, i) for i in os.listdir(os.path.join('data', cname)) if i.endswith('.wav')]
    data = [get_mfcc(file_path) for file_path in file_paths]
    all_data[cname] = data
    all_labels[cname] = [class_names.index(cname) for i in range(len(file_paths))]
    print((all_data[cname]))

[array([[-89.50311   , -78.347206  ,  -9.693185  , ...,   0.3601217 ,
          0.54920524,   0.51731783],
       [-89.50311   , -78.347206  ,  -9.693185  , ...,   0.3601217 ,
          0.54920524,   0.51731783],
       [-89.50311   , -78.347206  ,  -9.693185  , ...,   0.3601217 ,
          0.54920524,   0.51731783],
       ...,
       [-15.013916  ,   2.6160812 ,  30.689232  , ...,   0.9306584 ,
          0.87944597,   1.3138548 ],
       [-11.439941  ,   2.3877487 ,  20.932228  , ...,   0.9306584 ,
          0.87944597,   1.3138548 ],
       [-29.580688  , -18.312218  ,  10.014659  , ...,   0.9306584 ,
          0.87944597,   1.3138548 ]], dtype=float32), array([[-151.09622   ,  -67.236496  ,   18.219336  , ...,    0.41371602,
           1.6365294 ,    0.7462886 ],
       [-133.54391   ,  -63.4433    ,    4.7009144 , ...,    0.41371602,
           1.6365294 ,    0.7462886 ],
       [-132.64749   ,  -54.482117  ,    5.1316853 , ...,    0.41371602,
           1.6365294 ,    0.7462886 ]

[array([[-1.6767606e+02, -4.0210808e+01,  2.8515846e+01, ...,
         9.3618381e-01,  1.6553707e-01, -7.3170167e-01],
       [-1.6389212e+02, -3.7054070e+01,  2.3608274e+01, ...,
         9.3618381e-01,  1.6553707e-01, -7.3170167e-01],
       [-1.5818112e+02, -3.6182213e+01,  2.1186562e+01, ...,
         9.3618381e-01,  1.6553707e-01, -7.3170167e-01],
       ...,
       [-5.0730133e+01,  1.0347649e+01, -2.2585737e+01, ...,
         2.0763371e+00,  2.1726279e+00,  2.3224862e+00],
       [-6.2178955e+00, -4.5555840e+01, -7.3669834e+00, ...,
         2.0763371e+00,  2.1726279e+00,  2.3224862e+00],
       [ 4.0165649e+01, -5.3119652e+01,  4.6984792e+00, ...,
         2.0763371e+00,  2.1726279e+00,  2.3224862e+00]], dtype=float32), array([[-71.78516   , -19.123886  ,  -3.9803061 , ...,   0.35516915,
          0.35230199,   0.3315287 ],
       [-69.06213   , -15.29025   ,  -0.20119286, ...,   0.35516915,
          0.35230199,   0.3315287 ],
       [-68.38513   , -14.336382  ,   0.74077225, 

[array([[-241.9725    ,  -53.6669    ,   59.976334  , ...,   -0.9239642 ,
          -0.56927586,   -0.58899236],
       [-242.40536   ,  -57.01748   ,   51.20047   , ...,   -0.9239642 ,
          -0.56927586,   -0.58899236],
       [-249.3646    ,  -66.211395  ,   46.7555    , ...,   -0.9239642 ,
          -0.56927586,   -0.58899236],
       ...,
       [-182.14365   ,  -59.35843   ,   36.56476   , ...,    0.4051617 ,
          -1.0257218 ,   -0.47224066],
       [-184.79507   ,  -53.146698  ,   35.468563  , ...,    0.4051617 ,
          -1.0257218 ,   -0.47224066],
       [-196.43771   ,  -50.300392  ,   40.096603  , ...,    0.4051617 ,
          -1.0257218 ,   -0.47224066]], dtype=float32), array([[-2.2679492e+02, -3.9013809e+01,  6.8451050e+01, ...,
        -1.4960119e-01,  4.1857201e-01,  4.8916864e-01],
       [-2.2517664e+02, -3.7059555e+01,  6.7913292e+01, ...,
        -1.4960119e-01,  4.1857201e-01,  4.8916864e-01],
       [-2.2987689e+02, -3.9385086e+01,  7.2433823e+01, ...,
 

[array([[-273.79266   ,  -54.54493   ,   65.758896  , ...,   -0.3055706 ,
          -0.5983641 ,   -0.2922679 ],
       [-195.36945   ,  -45.20539   ,   24.944874  , ...,   -0.3055706 ,
          -0.5983641 ,   -0.2922679 ],
       [-226.99286   ,  -33.97329   ,   39.551716  , ...,   -0.3055706 ,
          -0.5983641 ,   -0.2922679 ],
       ...,
       [-131.74371   ,  -20.205788  ,   44.647903  , ...,   -1.0632298 ,
          -0.35437468,    0.41760263],
       [-156.8924    ,  -20.95855   ,   45.32139   , ...,   -1.0632298 ,
          -0.35437468,    0.41760263],
       [-180.07257   ,  -22.657227  ,   50.275585  , ...,   -1.0632298 ,
          -0.35437468,    0.41760263]], dtype=float32), array([[-1.1193741e+02, -7.7110794e+01,  4.0006065e+01, ...,
        -6.8371481e-01, -6.9330834e-02,  5.5846798e-01],
       [-9.5675201e+01, -6.9159462e+01,  1.8987202e+01, ...,
        -6.8371481e-01, -6.9330834e-02,  5.5846798e-01],
       [-9.9556183e+01, -7.0016190e+01,  3.4170883e+01, ...,
 

[array([[ -85.25461  ,  -23.48101  ,   15.430618 , ...,   -0.5015313,
           1.6511335,   -0.468184 ],
       [ -76.7392   ,   -8.443558 ,   24.280102 , ...,   -0.5015313,
           1.6511335,   -0.468184 ],
       [ -60.703796 ,  -22.422325 ,   12.047143 , ...,   -0.5015313,
           1.6511335,   -0.468184 ],
       ...,
       [-169.36002  ,  -57.35283  ,   51.72992  , ...,    2.169863 ,
           1.225302 ,    0.6166079],
       [ -48.95227  ,  -97.32848  ,   53.626312 , ...,    2.169863 ,
           1.225302 ,    0.6166079],
       [ -49.39972  ,  -94.99727  ,   50.780014 , ...,    2.169863 ,
           1.225302 ,    0.6166079]], dtype=float32), array([[ 4.30102539e+00, -9.64201355e+00,  4.45623398e-01, ...,
        -2.06692114e-01,  1.28652453e+00,  8.23043406e-01],
       [ 6.53517151e+01,  6.33636475e-01,  3.71710014e+00, ...,
        -2.06692114e-01,  1.28652453e+00,  8.23043406e-01],
       [ 1.93078476e+02, -1.89571991e+01,  3.01505547e+01, ...,
        -2.06692114e-0

In [4]:
from sklearn.model_selection import train_test_split

X = {'train': {}, 'test': {}}
y = {'train': {}, 'test': {}}
for cname in class_names:
    x_train, x_test, _, y_test = train_test_split(
        all_data[cname], all_labels[cname], 
        test_size = 0.2, 
        random_state=42
    )
    X['train'][cname] = x_train
    X['test'][cname] = x_test
    y['test'][cname] = y_test

In [5]:
print("Ten file", " Train", " Test")
for cname in class_names:
    print(" ", cname,"    ", len(X['train'][cname]), "   ",len(X['test'][cname]))

Ten file  Train  Test
  dup      28     7
  giu      40     11
  nha      37     10
  phai      28     8
  trai      39     10


In [6]:
import hmmlearn.hmm as hmm

model = {}
for idx, cname in enumerate(class_names):
    start_prob = np.full(states[idx], 0.0)
    start_prob[0] = 1.0
    trans_matrix = np.full((states[idx], states[idx]), 0.0)
    p = 0.5
    np.fill_diagonal(trans_matrix, p)
    np.fill_diagonal(trans_matrix[0:, 1:], 1 - p)
    trans_matrix[-1, -1] = 1.0
    #trans matrix
    model[cname] = hmm.GaussianHMM(
        n_components=states[idx], 
        verbose=True, 
        n_iter=300, 
        startprob_prior=start_prob, 
        transmat_prior=trans_matrix,
        params='stmc',
        init_params='mc',
        random_state=42
    )
    model[cname].fit(X=np.vstack(X['train'][cname]), lengths=[x.shape[0] for x in X['train'][cname]])
    print(model[cname].__dict__)

         1     -303337.9448             +nan
         2     -271361.4460      +31976.4988
         3     -260569.4769      +10791.9691
         4     -258251.9031       +2317.5738
         5     -257224.6465       +1027.2566
         6     -255984.9037       +1239.7428
         7     -254718.0306       +1266.8731
         8     -253956.7606        +761.2700
         9     -253692.4423        +264.3183
        10     -253495.1755        +197.2667
        11     -253307.3568        +187.8188
        12     -253131.7245        +175.6322
        13     -252930.3100        +201.4145
        14     -252727.8685        +202.4415
        15     -252540.9204        +186.9481
        16     -252330.0427        +210.8777
        17     -252116.9514        +213.0913
        18     -251795.3687        +321.5827
        19     -251410.4406        +384.9281
        20     -250722.8811        +687.5595
        21     -249964.1650        +758.7161
        22     -248595.1616       +1369.0033
        23

{'n_components': 12, 'params': 'stmc', 'init_params': 'mc', 'startprob_prior': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'transmat_prior': array([[0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. ]]), 'algorithm': 'vi

         1     -535914.0364             +nan
         2     -491041.8643      +44872.1722
         3     -475051.9599      +15989.9043
         4     -469911.0178       +5140.9421
         5     -466883.0101       +3028.0078
         6     -465097.7869       +1785.2232
         7     -464181.4598        +916.3271
         8     -463528.4821        +652.9777
         9     -463087.5920        +440.8901
        10     -462612.3593        +475.2326
        11     -462103.6921        +508.6672
        12     -461557.8609        +545.8312
        13     -460882.2123        +675.6486
        14     -459757.7103       +1124.5020
        15     -456639.7759       +3117.9344
        16     -452394.8796       +4244.8964
        17     -451132.9973       +1261.8823
        18     -451024.1810        +108.8163
        19     -450874.0711        +150.1100
        20     -450715.4057        +158.6653
        21     -450580.8014        +134.6044
        22     -450385.7966        +195.0048
        23

{'n_components': 19, 'params': 'stmc', 'init_params': 'mc', 'startprob_prior': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.]), 'transmat_prior': array([[0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. ,

         1     -394707.6689             +nan
         2     -367572.7714      +27134.8975
         3     -360773.5169       +6799.2545
         4     -358865.9621       +1907.5548
         5     -357860.6642       +1005.2979
         6     -357300.3559        +560.3083
         7     -356799.1433        +501.2126
         8     -356439.2826        +359.8607
         9     -356266.9664        +172.3163
        10     -356069.6728        +197.2935
        11     -355834.9657        +234.7072
        12     -355531.3932        +303.5725
        13     -355300.8751        +230.5181
        14     -354737.4762        +563.3989
        15     -354114.2675        +623.2087
        16     -352568.1127       +1546.1548
        17     -345613.6199       +6954.4928
        18     -343931.6368       +1681.9831
        19     -343931.4708          +0.1660
        20     -343931.4838          -0.0131


...

         1     -369752.9558             +nan
         2     -342359.9331      +27393.0227
         3     -332139.6496      +10220.2835
         4     -328037.7624       +4101.8872
         5     -326453.6228       +1584.1395
         6     -325190.9859       +1262.6369
         7     -323483.7684       +1707.2176
         8     -317301.0418       +6182.7265
         9     -309541.6226       +7759.4192
        10     -309344.7228        +196.8999
        11     -309193.4611        +151.2617
        12     -309105.4128         +88.0483
        13     -308936.6785        +168.7344
        14     -308783.5521        +153.1264
        15     -308712.1379         +71.4142
        16     -308672.1625         +39.9754
        17     -308639.4675         +32.6950
        18     -308614.3739         +25.0937
        19     -308595.0484         +19.3255
        20     -308583.9570         +11.0914
        21     -308577.4528          +6.5042
        22     -308570.3983          +7.0545
        23

{'n_components': 12, 'params': 'stmc', 'init_params': 'mc', 'startprob_prior': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'transmat_prior': array([[0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. ]]), 'algorithm': 'vi

         1     -558305.6127             +nan
         2     -510298.7316      +48006.8810
         3     -495178.0425      +15120.6891
         4     -490879.7898       +4298.2527
         5     -488122.2500       +2757.5398
         6     -486363.8227       +1758.4273
         7     -485118.4120       +1245.4107
         8     -484201.7049        +916.7071
         9     -483242.3389        +959.3660
        10     -482160.5694       +1081.7695
        11     -480630.8781       +1529.6913
        12     -478103.5897       +2527.2884
        13     -474226.6018       +3876.9880
        14     -468077.2592       +6149.3425
        15     -467970.0470        +107.2122
        16     -467937.4179         +32.6291
        17     -467912.2889         +25.1291
        18     -467898.1140         +14.1749
        19     -467891.8380          +6.2760
        20     -467888.7666          +3.0713
        21     -467886.9290          +1.8377
        22     -467885.4275          +1.5014
        23

{'n_components': 12, 'params': 'stmc', 'init_params': 'mc', 'startprob_prior': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'transmat_prior': array([[0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. ]]), 'algorithm': 'vi

        57     -467856.5551          +0.0211
        58     -467856.5411          +0.0141
        59     -467856.5315          +0.0096


In [7]:
import pickle

# save model
for cname in class_names:
    name = f'models_train\model_{cname}.pkl'
    with open(name, 'wb') as file: 
        pickle.dump(model[cname], file)

In [8]:
import pickle, os
import numpy as np

from sklearn.metrics import classification_report

In [9]:
y_true = []
y_pred = []
for cname in class_names:
    for mfcc, target in zip(X['test'][cname], y['test'][cname]):
        scores = [model[cname].score(mfcc) for cname in class_names]
        pred = np.argmax(scores)
        y_pred.append(pred)
        y_true.append(target)
    print((np.array(y_true) == np.array(y_pred)).sum()/len(y_true))
print(y_true)
print(y_pred)


0.8571428571428571
0.8888888888888888
0.7142857142857143
0.6944444444444444
0.7391304347826086
[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[4, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 4, 1, 2, 2, 4, 3, 3, 3, 1, 4, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4]


In [10]:
report = classification_report(y_true, y_pred, target_names=class_names)
print(report)

              precision    recall  f1-score   support

         dup       1.00      0.86      0.92         7
         giu       0.62      0.91      0.74        11
         nha       0.80      0.40      0.53        10
        phai       1.00      0.62      0.77         8
        trai       0.64      0.90      0.75        10

    accuracy                           0.74        46
   macro avg       0.81      0.74      0.74        46
weighted avg       0.79      0.74      0.73        46



In [11]:
#loadmodels
import pickle

model_train = {}
for key in class_names:
    name = f"models\model_{key}.pkl"
    with open(name, 'rb') as file:
        model_train[key] = pickle.load(file)

In [12]:
print(model_train)

{'dup': GaussianHMM(init_params='mc', n_components=3, n_iter=300, random_state=42,
            startprob_prior=array([1., 0., 0.]),
            transmat_prior=array([[0.5, 0.5, 0. ],
       [0. , 0.5, 0.5],
       [0. , 0. , 1. ]]),
            verbose=True), 'giu': GaussianHMM(init_params='mc', n_components=3, n_iter=300, random_state=42,
            startprob_prior=array([1., 0., 0.]),
            transmat_prior=array([[0.5, 0.5, 0. ],
       [0. , 0.5, 0.5],
       [0. , 0. , 1. ]]),
            verbose=True), 'nha': GaussianHMM(init_params='mc', n_components=6, n_iter=300, random_state=42,
            startprob_prior=array([1., 0., 0., 0., 0., 0.]),
            transmat_prior=array([[0.5, 0.5, 0. , 0. , 0. , 0. ],
       [0. , 0.5, 0.5, 0. , 0. , 0. ],
       [0. , 0. , 0.5, 0.5, 0. , 0. ],
       [0. , 0. , 0. , 0.5, 0.5, 0. ],
       [0. , 0. , 0. , 0. , 0.5, 0.5],
       [0. , 0. , 0. , 0. , 0. , 1. ]]),
            verbose=True), 'phai': GaussianHMM(init_params='mc', n_componen

In [13]:
def my_confusion_matrix(y_true, y_pred):
    N = np.unique(y_true).shape[0] # number of classes 
    print(N)
    cm = np.zeros((N, N))
    print(cm)
    for n in range(y_true.shape[0]):
        cm[y_true[n], y_pred[n]] += 1
    return cm 

In [18]:
import os
import numpy as np
import scipy.io.wavfile as wav
import matplotlib.pyplot as plt
from utils.estnoise_ms import * 
import math
import pyaudio
import wave
from array import array

FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
CHUNK = 1024
RECORD_SECONDS = 1
FILE_NAME="RECORDING.wav"

def _calculate_frequencies(audio_data,sr):
        data_freq = np.fft.fftfreq(len(audio_data),1.0/sr)
        data_freq = data_freq[1:]
        return data_freq

def _calculate_amplitude(audio_data):
        data_ampl = np.abs(np.fft.rfft(audio_data,axis=0))
        data_ampl = data_ampl[1:]
        return data_ampl

def _calculate_energy(data):
        data_amplitude = _calculate_amplitude(data)
        data_energy = data_amplitude ** 2
        return data_energy

def _connect_energy_with_frequencies(data_freq, data_energy):
        energy_freq = {}
        for (i, freq) in enumerate(data_freq):
            if abs(freq) not in energy_freq:
                energy_freq[abs(freq)] = data_energy[i] * 2
        return energy_freq

def _calculate_normalized_energy(data,sr):
        data_freq = _calculate_frequencies(data,sr)
        data_energy = _calculate_energy(data)
        #data_energy = self._znormalize_energy(data_energy) #znorm brings worse results
        energy_freq = _connect_energy_with_frequencies(data_freq, data_energy)
        return energy_freq

def _sum_energy_in_band(energy_frequencies, start_band, end_band):
        sum_energy = 0
        for f in energy_frequencies.keys():
            if start_band<f<end_band:
               sum_energy += energy_frequencies[f]
        return sum_energy


def count_THESHOLD():
    audio = pyaudio.PyAudio()

    stream = audio.open(format=FORMAT, channels=CHANNELS,
                rate=RATE, input=True,
                frames_per_buffer=CHUNK)
    print ("recording...")        
    frames = array('h')
    
    for i in range(0, int(RATE/CHUNK*1)):
              data = stream.read(CHUNK)
              data_chunk = array('h', data)
              frames.extend(data_chunk)
             
    stream.stop_stream()
    stream.close()
    audio.terminate()

    print("Done")

    energy_freq = _calculate_normalized_energy(frames, RATE) 
    
    
    sum_full_energy = sum(energy_freq.values()) / 42 
    
    
    return sum_full_energy

def record(THESHOLD):
    
    audio = pyaudio.PyAudio()

    stream = audio.open(format=FORMAT, channels=CHANNELS,
                rate=RATE, input=True,
                frames_per_buffer=CHUNK)
    print ("recording...")        
    frames = array('h')
    i = 0

    while True:
        data = stream.read(CHUNK)
        data_chunk = array('h', data)
        frames.extend(data_chunk)
        i += 1
        if i == 4:
            energy_freq = _calculate_normalized_energy(frames, RATE) 
            sum_full_energy = sum(energy_freq.values()) / 4
            if sum_full_energy / THESHOLD > 1.2:
                print(sum_full_energy)
                continue
            else:
                i = 0
                frames = array('h')
        if i == 86:
            break    

    stream.stop_stream()
    stream.close()
    audio.terminate()
     
    wavfile=wave.open(FILE_NAME,'wb')
    wavfile.setnchannels(CHANNELS)
    wavfile.setsampwidth(audio.get_sample_size(FORMAT)) 
    wavfile.setframerate(RATE)
    wavfile.writeframes(frames)
    wavfile.close() 
    

In [19]:
def predict():


        #Predict
        record_mfcc = get_mfcc("RECORDING.wav")
        scores = [model[cname].score(record_mfcc) for cname in class_names]
        predict_word = np.argmax(scores)
        print(scores)
        #class_names_vie = ['dup', 'giu', 'nha', 'phai', 'trai']
        print("Kết quả dự đoán: ", class_names[predict_word])   
        return class_names[predict_word]  
        
    

In [24]:
from pynput.mouse import Button, Controller
import time

mouse = Controller()
while True:
    record(count_THESHOLD())
    if (predict()=="trai"):
        mouse.press(Button.left)
        mouse.release(Button.left)
    elif (predict()=="phai"):
        mouse.press(Button.right)
        mouse.release(Button.right)
    elif (predict()=="nha"):
        mouse.release(Button.left)
    elif (predict()=="giu"):
        mouse.press(Button.left)
    elif(predict()=="dup") :
        mouse.click(Button.left, 2)

recording...
Done
recording...
307204506309391.2
[-16691.533677615495, -16569.292440564506, -16263.694683718291, -16635.492580938575, -16295.897230205543]
Kết quả dự đoán:  nha
[-16691.533677615495, -16569.292440564506, -16263.694683718291, -16635.492580938575, -16295.897230205543]
Kết quả dự đoán:  nha
[-16691.533677615495, -16569.292440564506, -16263.694683718291, -16635.492580938575, -16295.897230205543]
Kết quả dự đoán:  nha
recording...
Done
recording...
278820265315005.5
[-16348.61658503393, -17165.792313314705, -16665.940088066764, -16256.913788414886, -16122.798121391166]
Kết quả dự đoán:  trai
recording...
Done
recording...
357714445111408.56
[-15926.14496770248, -16606.16108226754, -15911.372583100658, -15458.39953195249, -15534.65704969029]
Kết quả dự đoán:  phai
[-15926.14496770248, -16606.16108226754, -15911.372583100658, -15458.39953195249, -15534.65704969029]
Kết quả dự đoán:  phai
recording...
Done
recording...
327152057527183.44
[-16852.734966221917, -17063.01260963412

KeyboardInterrupt: 